In [2]:
import requests
import time
import pandas as pd
import numpy as np
import uuid

pd.options.mode.chained_assignment = None

In [3]:
FOLDER = './'

def build_payload(limit):
    PAYLOAD = 'https://blocks.flashbots.net/v1/blocks' \
             + f'?limit={limit}'
    return PAYLOAD

def send_payload(payload):
    try:
        response = requests.get(payload)

        raw = response.json()
        raw = raw['blocks']

    except:
        time.sleep(59)
        return send_payload(payload)

    return raw

In [4]:
payload = build_payload(5)

raw = send_payload(payload)

df_block = pd.DataFrame.from_dict(raw)

df_block.drop(columns=['transactions'], inplace=True)
df_block.sort_values(by='block_number', ascending=True, inplace=True)
df_block.reset_index(drop=True, inplace=True)
#df_raw.to_csv('blocks.csv', mode='a', index=False)


def create_transactions(raw):
    txn = []
    for i in range(0, len(raw)):
        txn.append(raw[i]['transactions'])
    
    txn = [item for sublist in txn for item in sublist]
    return txn

txn = create_transactions(raw)
df_txn = pd.DataFrame.from_dict(txn)

#df_txn.to_csv('txn.csv', mode='a', index=False)








In [ ]:
def create_bundles(df_txn):
    blocks = np.unique(df_txn['block_number'])
    bdl = []
    for i in range(0, len(blocks)):
        bundles = np.unique(df_txn[df_txn['block_number'] == blocks[i]]['bundle_index'])
        for j in range(0, len(bundles)):
            temp = df_txn[(df_txn['block_number'] == blocks[i]) & (df_txn['bundle_index'] == bundles[j])]
            temp['id'] = uuid.uuid4()
            temp.reset_index(drop=True, inplace=True)
            bdl.append(temp.to_dict())

    return bdl

bdl = create_bundles(df_txn)
bdl_df = pd.DataFrame.from_dict(bdl)